In [1]:
""" Importing necessary libraries for SVD microarray analysis """

# %pip install numpy==2.3.0 pandas==2.3.0 matplotlib==3.10.3 scipy==1.15.3 ncbi-datasets-pylib==16.6.1 GEOparse==2.0.4

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.linalg import svd
from scipy.sparse import eye as sparse_eye
from scipy.linalg import solve

from ncbi.datasets import GeneApi # Not used
import GEOparse as gp # Not used but could be useful for full streamlining the process

In [2]:
""" Loading the microarray data """

def load_from_geneapi():
    """ Function to load data from NCBI Gene API """

    gene_api = GeneApi()
    print(type(gene_api))

def load_from_geoparse():
    """ Function to load data from GEOparse """

    # Example of loading a GEO dataset
    gse = gp.get_GEO("GSE27011")  # Replace with actual GEO accession number
    print(type(gse))

def load_from_manual_selected_series():
    """ Function to load manually selected series data """
    # Read data from a CSV file
    path = "../Data/manual_series_splitting.csv"
    df = pd.read_csv(path, delimiter=',', header=0)  # Replace with actual file path
    # print(df.head())
    return df


MatrizNCBI = load_from_manual_selected_series()

In [ ]:
""" Primeira Etapa - Decomposição """

# Escolher um microarray dentre os disponíveis no NCBI. Fazer o download da matriz, caracterizando o problema subjacente ao microarray como um vector space model,
# onde se deseja resolver problemas de classificação. Descrever as entidades e o tipo de atributos utilizados. Resolver eventuais omissões como os atributos cujos valores não foram informados.

# MatrizNCBI deve estar carregada como um np.ndarray (genes x amostras)
# Seleção das colunas da matriz: aqui estou selecionando apenas os pacientes
# controle (sem asma, do 1 ao 19) e os pacientes com asma severa (39 ao final), excluo os pacientes com asma moderada
Ans = np.concatenate((MatrizNCBI[:, :19], MatrizNCBI[:, 38:]), axis=1)

# SVD (decomposição em valores singulares) - usar os componentes para visualizar variância e calcular centróides,
# é uma etapa de exploração dimensional.

Ans = Ans.T

Ans = Ans - np.mean(Ans, axis=0)  # Subtrai a média de cada coluna (gene)

T, S_diag, Vt = svd(Ans, full_matrices=False)
S = np.diag(S_diag)

diagonal_S = S_diag
dist_import_relativa = diagonal_S / np.sum(diagonal_S)

plt.figure()
plt.grid(True)
plt.plot(dist_import_relativa, '*')
plt.plot(dist_import_relativa)
plt.show()

# A matriz PC_scores representa as entidades (amostras) projetadas no novo espaço dos PCs (35x35).
PC_scores = T @ S  # PC_scores será 35x35

# Esses "scores" são as representações das suas 35 amostras em um novo espaço de menor dimensão,
# onde os dados estão reorganizados para capturar a maior variação possível nos primeiros componentes (tipo PCA).

# Labels: 0 para controle, 1 para asma severa
labels = np.concatenate((np.zeros(19), np.ones(17)))

# Índices
controle_idx = (labels == 0)
asma_idx = (labels == 1)

# Cálculo dos centróides nos 3 primeiros PCs
centroide_controle = PC_scores[controle_idx, :3].mean(axis=0)
centroide_asma = PC_scores[asma_idx, :3].mean(axis=0)

# Distância entre centróides
distancia_centroide = np.linalg.norm(centroide_controle - centroide_asma)
print(f'Distância entre centróides: {distancia_centroide:.4f}')

# ANOVA

group = np.array(['controle'] * 19 + ['asma'] * 17)
anova_result = stats.f_oneway(
    PC_scores[controle_idx, 0], PC_scores[asma_idx, 0])
print(f'p-valor da ANOVA no PC1: {anova_result.pvalue:.4e}')


In [ ]:
""" Etapa 2 """
# Construção de um Classificador: usar um modelo de regressão logística modificada para proceder à classificação dos problemas associados, considerando a matriz de entidades completa.

Aux = S @ Vt
x = Aux[0, :]
y = Aux[1, :]
z = Aux[2, :]

matriz_3x36 = np.array([x, y, z])

# Criação do gráfico
plt.figure()
plt.title('Projeção das Amostras no Espaço PC1 x PC2')
plt.xlabel('Primeiro Componente Principal (PC1)')
plt.ylabel('Segundo Componente Principal (PC2)')
plt.grid(True)

plt.scatter(x[labels == 0], y[labels == 0], c='r', label='Controle')
plt.scatter(x[labels == 1], y[labels == 1], facecolors='none',
            edgecolors='r', label='Asma severa')
plt.legend()
plt.show()

# Visualização antes da regressão logística modificada

Ans_Teste = np.concatenate((MatrizNCBI[:, :19], MatrizNCBI[:, 38:]), axis=1).T
# Centraliza com média do Ans original
Ans_Teste = Ans_Teste - np.mean(Ans, axis=0)

U, S2_diag, Vt = svd(Ans_Teste, full_matrices=False)
S2 = np.diag(S2_diag)
PC_scores_Teste = U @ S2  # Projeção

xt = PC_scores_Teste[:, 0]
yt = PC_scores_Teste[:, 1]

plt.figure()
plt.title('Projeção das Amostras no Espaço PC1 x PC2 (via SVD)')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.grid(True)
plt.scatter(xt[labels == 0], yt[labels == 0], c='r', label='Controle')
plt.scatter(xt[labels == 1], yt[labels == 1], facecolors='none',
            edgecolors='k', linewidths=1.2, label='Asma severa')
plt.legend()
plt.show()

# ======
# Selection of the 10 most important markers


def solve_system(A, b):
    m, n = A.shape
    Im = sparse_eye(m).toarray()
    In = sparse_eye(n).toarray()
    M = np.block([[Im, -A], [-A.T, In]])
    nb = np.zeros(m + n)
    nb[:m] = -b
    x = np.linalg.solve(M, nb)
    alpha = x[m:]
    return alpha, x


# Modified logistic regression thresholds
logit_ch1 = 12
logit_ch0 = -12

DataMatrix = Ans.T  # genes x amostras (35 amostras)

m, n = DataMatrix.shape
b = np.zeros(n)
b[:19] = logit_ch1
b[19:] = logit_ch0

alpha, _ = solve_system(DataMatrix.T, b)

plt.figure()
plt.title('Weights Associated with Attributes')
plt.plot(alpha, '*')
plt.show()

aux = DataMatrix.T @ alpha
num = np.exp(aux)
p = num / (1 + num)

plt.figure()
plt.title('Logistic Regression Classification - All Attributes')
plt.plot(p, '*')
plt.show()

# Validate the markers
positions = np.argsort(alpha)
selected = np.concatenate((positions[:7], positions[-7:]))

ReducedMatrix = DataMatrix[selected, :]

T, S_diag, Vt = svd(ReducedMatrix, full_matrices=False)
singular_values = S_diag
relative_importance = singular_values / np.sum(singular_values)

plt.figure()
plt.title('Relative Singular Values of the Reduced Matrix')
plt.grid(True)
plt.plot(relative_importance, '*')
plt.plot(relative_importance)
plt.show()

AuxiliaryMatrix = np.diag(singular_values) @ Vt
x = AuxiliaryMatrix[0, :]
y = AuxiliaryMatrix[1, :]
z = AuxiliaryMatrix[2, :]

plt.figure()
plt.title('Entity Domain Visualization')
plt.grid(True)
plt.plot(x, y, 'or')
plt.plot(x[:20], y[:20], '*r')
plt.show()

# P(x) for the reduced matrix

new_alpha = np.linalg.lstsq(ReducedMatrix.T, b, rcond=None)[0]

plt.figure()
plt.title('Weights Associated with Selected Attributes')
plt.plot(new_alpha, '*')
plt.show()

aux = ReducedMatrix.T @ new_alpha
num = np.exp(aux)
p = num / (1 + num)

plt.figure()
plt.title('Logistic Regression Classification - Selected Attributes')
plt.plot(p, '*')
plt.show()